In [1]:
!mamba install tensorflow-gpu==2.11.0 -y -q
!mamba install pytorch-cuda=11.6 -c pytorch -c conda-forge -c nvidia -y -q
!mamba install -c conda-forge pyts==0.12.0 -y -q
!pip install torch==1.13 torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116 -q
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.1+cu116.html -q
!pip install ts2vg==1.2.1 -q
!pip install pytorch_lightning==1.9.1 -q
!pip install torchsummary==1.5.1 -q
!pip install dvclive==2.0.2 -q

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... By downloading and using the CUDA Toolkit conda packages, you accept the terms and conditions of the CUDA End User License Agreement (EULA): https://docs.nvidia.com/cuda/eula/index.html

By downloading and using the cuDNN conda packages, you accept the terms and conditions of the NVIDIA cuDNN EULA -
  https://docs.nvidia.com/deeplearning/cudnn/sla/index.html

done
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


In [25]:
import os
import numpy as np
import random

from pyts.image import MarkovTransitionField

from ts2vg import NaturalVG
from ts2vg import HorizontalVG

from torch_geometric.data import Data

def get_versions_TSSB():
    """
    Gets the versions of TSSB

    Affected:
        Config["graph"]["dataset_path"]


    Returns:
       versions: a list of possible utilities for TSSB
    """
    versions = []
    directory = Config["graph"]["dataset_path"]

    if os.path.isdir(directory):
        versions = os.listdir(directory)
        versions = [file for file in versions if file not in ['desc.txt', 'properties.txt']]
        versions = sorted([version for version in versions if not version.startswith('.')])
    
    return versions

def get_TSSB(name_of_X):
    """
    Gets the X and mask of a chosen TSSB time series

    Args:
        name_of_X: name of the chosen TS we want and its mask
    
    Returns:
       _X: a 1D array containing the time steps of the chosen TSSB TS
       mask: a 1D array containing the mask for the chosen TSSB TS
    """
    
    desc_file = os.path.join(Config["graph"]["dataset_path"], "desc.txt")
    ts_file = os.path.join(Config["graph"]["dataset_path"], name_of_X)
    
    print(ts_file)
    data = {}
    
    # Read the description file and populate the data dictionary
    with open(desc_file) as f:
        for line in f:
            row = line.strip().split(",")
            name = row[0]
            values = [int(value) for value in row[2:]]
            data[name] = values
            
    # Load the time series data from the specified file and create an array of zeros with the same length as _X
    _X = np.loadtxt(ts_file)
    mask = np.zeros(len(_X))
    
    # Check if the name_of_X (without the file extension) is in the data dictionary
    if name_of_X[:-4] in data:
        for value in data[name_of_X[:-4]]:
            if value < len(mask):
                mask[value] = 1

    return _X, mask

def transform_mask(_X, _mask):
    """
    Transforms the mask from the shape of, for example, [0,0,1,0,0,0,1,0,0,1,0,1,0,0,0] to [0,0,1,1,1,1,2,2,2,3,3,4,4,4,4] and repairs it to [0,0,1,1,1,1,2,2,2,3,3,1,1,1,1] if 1 and 4 are the same segments

    Args:
        _X: a 1D array containing time series, used to check if _X is repeating anywhere inside it
        _mask: a 1D array that will be converted from 1 and 0 to a multi-label mask
    
    Returns:
        new_mask: a 1D array where the mask is converted from 1 and 0 to a multi-label mask, mostly used for time series segmentation
    """
    current_group = 0
    group_indices = []
    
    # Iterate over the elements in the _mask array
    for i in range(len(_mask)):
        if _mask[i] == 1:
            current_group += 1
        group_indices.append(current_group)
    new_mask = np.array(group_indices)

    unique_values = np.unique(new_mask)
    total_unique_values = np.arange(len(unique_values))
    
    # Iterate over the unique values in the new_mask array
    for i in unique_values:
        for j in range(i, unique_values[-1] + 1):
            is_equal = np.all(_X[new_mask == i][:100] == _X[new_mask == j][:100])
            if is_equal:
                total_unique_values[j] = total_unique_values[i]
                
    # Update the new_mask array using the total_unique_values array
    for i in unique_values:
        new_mask[new_mask == i] = total_unique_values[i]

    true_unique_values = np.unique(new_mask)
    true_total_unique_values = np.arange(len(true_unique_values))
    
    # Update the new_mask array with the true_total_unique_values array
    for i in true_total_unique_values:     
        mask_index = new_mask == true_unique_values[i]
        new_mask[mask_index] = int(i)
        
    return new_mask.reshape(1, -1)


def get_matrix(X_current):
    """
    This function gets the adjacency matrix through either visibility, MTF, or dual VG graph

    Args:
        X_current: a 1D array usually containing time series values
    
    Returns:
        adj_mat: a list of adjacency matrices
    """
    adj_mat = []
    
    # Check the graph type specified in the Config and perform the corresponding operations
    if Config["graph"]["type"] in ("VG", "Dual_VG"):
        VGConfig = Config["graph"]["VG"]
        
        # Create an instance of the visibility graph class based on the edge type specified
        if VGConfig["edge_type"] == "natural":
            g = NaturalVG(weighted=VGConfig["distance"])
        elif VGConfig["edge_type"] == "horizontal":
            g = HorizontalVG(weighted=VGConfig["distance"])
        
        # Build the visibility graph using the provided time series
        g.build(X_current)

        adj_mat_vis = np.zeros([len(X_current), len(X_current)], dtype='float')
        
        # Iterate over the edges of the visibility graph and assign weights to the corresponding positions in the adjacency matrix
        for x, y, q in g.edges:
            adj_mat_vis[x, y] = q
            if VGConfig["edge_dir"] == "undirected":
                adj_mat_vis[y, x] = q
        
        adj_mat.append(adj_mat_vis)
        
    elif Config["graph"]["type"] == "MTF":
        n_bins = Config["graph"]["MTF"]["num_bins"]
        if n_bins == "auto":
            n_bins = int(len(X_current)/2)

        # Create and compute an instance of the Markov Transition Field class
        MTF = MarkovTransitionField(n_bins=n_bins)
        X_gaf_MTF_temp = MTF.fit_transform(X_current.reshape(1, -1))
        adj_mat.append(X_gaf_MTF_temp[0])
    
    return adj_mat
    
def adjToEdgidx(adj_mat):
    """
    This function creates edge indexes and weights for a given matrix
    
    Args:
        adj_mat: a 2D array

    Returns:
        edge_index: a 2D torch array that indicates the connected values
        edge_weight: a 1D array of weights that represent the absolute distance between connected nodes or values in the time series
    """
    edge_index = torch.from_numpy(adj_mat[0]).nonzero().t().contiguous()
    row, col = edge_index
    edge_weight = adj_mat[0][row, col]
    return edge_index, edge_weight

def adjToEdgidx_Dual_VG(X_current):
    """
    Creates a dual visibility graph by first creating a directed VG from one side and then flipping and running the get_matrix function again.
    By doing this, we join these two graphs and obtain a dual VG.

    Args:
        X_current: 1D array usually containing time series values

    Returns:
        edge_index: 2D torch array that defines the connected values
        edge_weight: 2D array of weights that represent the absolute distance between every node or value in the time series
    """
    pos_adj_mat_vis = get_matrix(X_current)[0]
    neg_adj_mat_vis = get_matrix(-X_current)[0]
    edge_index = torch.from_numpy(pos_adj_mat_vis + neg_adj_mat_vis).nonzero().t().contiguous()

    # Join two edge_weight arrays
    row, col = edge_index
    edge_weight = np.zeros([len(row), 2], dtype='float')
    edge_weight[:, 0] = pos_adj_mat_vis[row, col]
    edge_weight[:, 1] = neg_adj_mat_vis[row, col]

    return edge_index, edge_weight

def create_mask(train, val, test, max_size):
    """
    Generates masks for train, validation, and test sets based on specified percentages and a maximum size.

    Args:
        train: float value representing the percentage of the train set
        val: float value representing the percentage of the validation set
        test: float value representing the percentage of the test set
        max_size: integer representing the maximum size of the masks

    Returns:
        train_mask: boolean array indicating the train set
        val_mask: boolean array indicating the validation set
        test_mask: boolean array indicating the test set
    """
    if train + val + test != 1:
        print("train, val, and test do not add up to 1")
    else:
        random.seed(Config['model']['SEED'])
        percentages = [train, val, test]  # Percentage of each value
        values = [1, 2, 3]  # Values to use
        counts = [int(max_size * p) for p in percentages]  # Count of each value
        counts[-1] += max_size - sum(counts)  # Adjust for rounding errors

        lst = []
        for i, count in enumerate(counts):
            lst.extend([values[i]] * count)

        random.shuffle(lst)

        train_mask = np.array([x == 1 for x in lst])
        val_mask = np.array([x == 2 for x in lst])
        test_mask = np.array([x == 3 for x in lst])

        return train_mask, val_mask, test_mask
    
def create_graph(output, X, mask):
    """
    Creates a graph in the torch geometric Data format, containing the node values x, mask values for training, testing, and validation, edge indexes, and edge attributes.

    Args:
        output: Dataset of multiple graphs (optional). New graph will be appended to this dataset.
        X: Node values (integer).
        mask: 1D array representing the mask.

    Returns:
        output: Updated dataset of multiple or singular graph.
    """
    if Config["graph"]["type"] in ("VG", "MTF"):
        edge_index, edge_weight = adjToEdgidx(get_matrix(X))
    elif Config["graph"]["type"] == "Dual_VG":
        edge_index, edge_weight = adjToEdgidx_Dual_VG(X)

    x = torch.unsqueeze(torch.tensor(X, dtype=torch.double), 1).clone().detach()
    edge_index = edge_index.clone().detach().to(torch.int64)
    edge_attr = torch.unsqueeze(torch.tensor(edge_weight, dtype=torch.double), 1).clone().detach()
    y = torch.unsqueeze(torch.tensor(mask[0], dtype=torch.double), 1)

    train_mask, val_mask, test_mask = create_mask(Config["model"]["train/val/test"]["train"], Config["model"]["train/val/test"]["val"], Config["model"]["train/val/test"]["test"], len(X))
    train_mask = torch.tensor(train_mask, dtype=torch.bool)
    val_mask = torch.tensor(val_mask, dtype=torch.bool)
    test_mask = torch.tensor(test_mask, dtype=torch.bool)

    output.append(Data(x=x, train_mask=train_mask, val_mask=val_mask, test_mask=test_mask, edge_index=edge_index, edge_attr=edge_attr, y=y))
    return output

In [7]:
from sklearn.utils import class_weight
def TSSB_graph():
    """
    Performs a pipeline of operations to process the TSSB dataset for a given utility.

    Returns:
        output: A list containing the processed graph data.
        class_weights: Torch tensor containing the computed class weights.
    """
    # gets all time series names 
    versions = get_versions_TSSB()
    
    # gets the X, lables for the selected utility 
    X, mask = get_TSSB(versions[utility])
    mask = transform_mask(X, mask)
    
    # creates a graph from the selected time series
    dataset = create_graph([], X, mask)
    
    #initiates class weights for all diferent segments in the time series
    class_weights = torch.tensor(class_weight.compute_class_weight(class_weight='balanced',
                                                                    classes=np.unique(mask[0]),
                                                                    y=mask[0]))

    return dataset, class_weights


In [29]:
utility=74
TSSB_graph()

datasets/TSSB/Yoga.txt


([Data(x=[15974, 1], edge_index=[2, 63992], edge_attr=[63992, 1], y=[15974, 1], train_mask=[15974], val_mask=[15974], test_mask=[15974])],
 tensor([1.0949, 0.9203], dtype=torch.float64))

In [9]:
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from dvclive.lightning import DVCLiveLogger
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATConv
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

class GAT(pl.LightningModule):
    def __init__(self, class_weights, model,Config):
        """
        Initializes the GAT model.

        Args:
            class_weights: The class weights used for training.
        """
        super(GAT, self).__init__()
        
        self.class_weights = class_weights
        self.model = model
        self.Config = Config
        
        self.conv1 = GATConv(1, 32, heads=4)
        self.conv2 = GATConv(4 * 32, 32, heads=4)
        self.conv3 = GATConv(4 * 32, 32, heads=8)
        self.conv4 = GATConv(8 * 32, len(self.class_weights), heads=6,concat=False)


    def forward(self, data):
        """
        Performs the forward pass of the model.

        Args:
            data: The input data.

        Returns:
            x: The output of the model.
        """
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr

        x = F.elu(self.conv1(x, edge_index, edge_weight))
        x = F.elu(self.conv2(x, edge_index, edge_weight))
        x = F.elu(self.conv3(x, edge_index, edge_weight))
        x = self.conv4(x, edge_index, edge_weight)
        return x

    
    def configure_optimizers(self):
        """
        Configures the optimizer for training the model.

        Returns:
            optimizer: The configured optimizer.
        """
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.Config["model"]["learning_rate"], weight_decay=5e-4)
        return optimizer
    
    def training_step(self, data, batch_idx):
        """
        Performs a single training step on the given batch of train data.

        Args:
            data: Input data for the training step.
            batch_idx: Index of the current batch.

        Returns:
            train_loss: Loss value for the training step.
        """
        out = self.model(data)
        loss_function = CrossEntropyLoss(weight=self.class_weights).to(self.device)
        train_loss = loss_function(out[data.train_mask], data.y[data.train_mask].squeeze().to(torch.int64))
        
        return train_loss

    def test_step(self, data, batch_idx):
        """
        Performs a forward pass on the model to obtain predictions for the test data. It calculates the test loss, accuracy, and collects the true labels and predicted labels for later evaluation.

        Args:
            data: Test data for the current batch.
            batch_idx: Index of the current batch.

        Returns:
            pred: Predicted labels for the test data.
            y: True labels for the test data.
        """
        out = self.model(data)
        loss_function = CrossEntropyLoss(weight=self.class_weights).to(self.device)
    
        test_loss = loss_function(out[data.test_mask], data.y[data.test_mask].squeeze().to(torch.int64))

        ys, preds = [], []
        test_label = data.y[data.test_mask].cpu()
        ys.append(data.y[data.test_mask])
        preds.append((out[data.test_mask].argmax(-1)).float().cpu())

        y, pred = torch.cat(ys, dim=0), torch.cat(preds, dim=0)
        pred = pred.reshape(-1, 1)
        accuracy = (pred == test_label).sum() / pred.shape[0]

        self.log("test_loss", test_loss)
        self.log("test_acc", accuracy)

        return pred, y.squeeze()


    def test_epoch_end(self, outputs):
        """
        Test epoch end function.

        This function receives accumulated predicted and true labels from the test_step and uses them on the confusion matrix and classification report.

        Args:
            pred: Predicted labels for the test data.
            y: True labels for the test data.
        
        Returns:
            prints a confusion matrix and a classification report

        """
        global true_array, pred_array
        true_array = np.concatenate([output[1].cpu().numpy() for output in outputs], axis=0)
        pred_array = np.concatenate([output[0].cpu().numpy() for output in outputs], axis=0)

        print(confusion_matrix(true_array, pred_array))
        print(classification_report(true_array, pred_array))

        
def main():
    """
    Main function for training.
    
    Affected by:
        Config["model"]
    """
    global device
    
    # initiate callback functions, DVC, Seed and device
    early_stop = EarlyStopping(monitor='val_loss',patience=Config["model"]["patience"], strict=False,verbose=False, mode='min')
    val_checkpoint_best_acc = ModelCheckpoint(filename="best_acc", monitor = "val_acc", mode="max")
    val_checkpoint_best_loss = ModelCheckpoint(filename="best_loss", monitor = "val_loss", mode="min")
    logger = DVCLiveLogger(run_name = Config["model"]["name_of_save"])    
    
    torch.manual_seed(Config['model']['SEED'])
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # creates dataset containing graphs and the overall class_weights
    output, class_weights = TSSB_graph()
    #create a loader. Only one is needed as train and test masks are used
    loader = DataLoader(output, batch_size=1, shuffle=False)

    # initializes the model
    model = GAT(class_weights, model,Config).double()

    #traines the model arhitecture
    trainer = pl.Trainer(logger=logger, max_epochs = Config["model"]["range_epoch"], callbacks=[val_checkpoint_best_acc,val_checkpoint_best_loss,early_stop],accelerator='gpu',devices=1)
    trainer.fit(model, loader)
    
    #tests the model arhitecture and prints the results
    tester = pl.Trainer(accelerator='gpu',devices=1)
    tester.test(model, loader)

In [10]:
def cut_the_peaks(pred_array, percentage_num): # CHANGE POINT DETECTION
    """
    Cuts the peaks in the given prediction array based on a specified percentage threshold.

    Args:
        pred_array: Array containing predictions.
        percentage_num: Percentage threshold to determine when to cut the peaks.

    Returns:
        array1: Array after cutting the peaks in the first iteration.
        array2: Array after cutting the peaks in the second iteration.
        array3: Array after cutting the peaks in the third iteration.
        array4: Array after cutting the peaks in the fourth iteration.
        array5: Array after cutting the peaks in the fifth iteration.
    """
    _pred_array = pred_array.copy()
    len_array = len(_pred_array)

    def array_stats(arr):
        """
        Calculates the most frequent value and its frequency percentage in the given array.

        Args:
            arr: Input array.

        Returns:
            most_frequent: Most frequent value in the array.
            percentage: Frequency percentage of the most frequent value.
        """
        values, counts = np.unique(arr, return_counts=True)
        most_frequent = values[np.argmax(counts)]
        frequency = np.max(counts)
        percentage = (frequency / len(arr)) * 100
        return most_frequent, percentage

    def initial_while_loop(i, array, percentage_num):
        """
        Applies the peak cutting process in a while loop for a specified section of the array.

        Args:
            i: Index to define the section of the array.
            array: Array to be processed.
            percentage_num: Percentage threshold to determine when to cut the peaks.

        Returns:
            array: Array after cutting the peaks in the specified section.
        """
        count = 0 + i * 5
        array = array.copy()
        while count < len_array - 10:
            temp_array = array[count : count + 30]
            most_frequent, percentage = array_stats(temp_array)
            if percentage > percentage_num:
                array[count : count + 30] = np.array([most_frequent] * 30)[0]
            count += 30
        return array

    array1 = initial_while_loop(0, _pred_array, percentage_num)
    array2 = initial_while_loop(1, _pred_array, percentage_num)
    array3 = initial_while_loop(2, _pred_array, percentage_num)
    array4 = initial_while_loop(3, _pred_array, percentage_num)
    array5 = initial_while_loop(4, _pred_array, percentage_num)

    return array1, array2, array3, array4, array5


def merge_arrays(arr1, arr2, arr3, arr4, arr5):
    """
    Merges multiple arrays based on certain conditions.

    Args:
        arr1: First input array.
        arr2: Second input array.
        arr3: Third input array.
        arr4: Fourth input array.
        arr5: Fifth input array.

    Returns:
        merged_arr: Merged array after applying the merging conditions.
    """
    merged_arr = []
    for i in range(len(arr1)):
        if (
            (arr1[i] == arr2[i] == arr3[i])
            or (arr1[i] == arr2[i] == arr4[i])
            or (arr1[i] == arr2[i] == arr5[i])
            or (arr1[i] == arr3[i] == arr4[i])
            or (arr1[i] == arr3[i] == arr5[i])
            or (arr1[i] == arr4[i] == arr5[i])
        ):
            merged_arr.append(arr1[i])
        elif (arr2[i] == arr3[i] == arr4[i]) or (arr2[i] == arr3[i] == arr5[i]) or (
            arr2[i] == arr4[i] == arr5[i]
        ):
            merged_arr.append(arr2[i])
        elif arr3[i] == arr4[i] == arr5[i]:
            merged_arr.append(arr3[i])
        else:
            merged_arr.append(merged_arr[i - 1])
    return np.array(merged_arr)


def check_on_right_and_left(array):
    """
    Checks the values on the right and left of each element in the array and replaces the element based on certain conditions.

    Args:
        array: Input array.

    Returns:
        array: Array after replacing elements based on the conditions.
    """
    for i in range(6, len(array) - 6):
        if array[i - 1] == array[i + 1]:
            array[i] = array[i + 1]
        if (
            array[i - 3] == array[i - 2]
            == array[i + 2] == array[i + 3]
        ):
            array[i] = array[i + 2]
        if (
            array[i - 6] == array[i - 5]
            == array[i + 5] == array[i + 6]
        ):
            array[i] = array[i + 5]
    return array


def get_change(array):
    """
    Determines the change points in the given array.

    Args:
        array: Input array.

    Returns:
        mask_temp: Array indicating the change points (1) and non-change points (0).
    """
    array = array[10:]
    temp = array[11]
    mask_temp = [temp] * 10
    mask_temp = np.array(mask_temp).reshape(-1)

    for i in range(len(array)):
        if array[i] != temp:
            temp = array[i]
            mask_temp = np.append(mask_temp, np.array([1]))
        else:
            mask_temp = np.append(mask_temp, np.array([0]))

    mask_temp = np.array(mask_temp)
    return mask_temp


def plt_peaks():
    """
    Plots the peaks and change points for visualization.
    """
    # Cut peaks 1
    array1, array2, array3, array4, array5 = cut_the_peaks(pred_array, 80)
    merged_array1 = merge_arrays(array1, array2, array3, array4, array5)
    # Cut peaks 2
    merged_array1 = check_on_right_and_left(merged_array1)
    # Cut peaks 3
    array1, array2, array3, array4, array5 = cut_the_peaks(merged_array1, 70)
    merged_array2 = merge_arrays(array1, array2, array3, array4, array5)
    # Show where the change occurs
    mask_pred = get_change(merged_array2)
    mask_true = get_change(true_array)

    fig, [ax1, ax2, ax3, ax4] = plt.subplots(ncols=1, nrows=4, figsize=(20, 5))

    ax1.plot(pred_array)
    ax1.plot(true_array, color="yellow")
    ax2.plot(mask_pred * 2)
    ax2.plot(mask_true, color="yellow")
    ax3.plot(loader.dataset[0].y)
    ax4.plot(loader.dataset[0].train_mask)

In [11]:
"""
This list contains configurations for generating and training a graph from a TSSB time series.
"""

Config = {  
    "graph": {
        "dataset_path": "datasets/TSSB/", # path to the TSSB dataset. can be downloaded from: https://github.com/ermshaua/time-series-segmentation-benchmark/tree/main/tssb/datasets
        "type": "MTF",  # Type of graph (MTF, VG, Dual_VG)
        "MTF": {
            "num_bins": "auto"  # Number of bins for MTF graph (integer or "auto")
        },
        "VG": {
            "edge_type": "natural",  # Type of edge calculation for VG graph (natural or horizontal)
            "distance": 'distance',  # Type of distance metric for VG graph (slope, abs_slope, distance, h_distance, v_distance, abs_v_distance)
            "edge_dir": "directed"  # Directionality of edges in VG graph (undirected or directed)
        }
    },
    
    "model": {
        "SEED": 820,  # Random seed for reproducibility
        "learning_rate": 0.005,  # Learning rate for training
        "batch_size": 64,  # Batch size for training
        "range_epoch": 2000,  # Number of training epochs
        "save_file": "test_test",  # File name for saving trained model
        "name_of_save": "test_u-time",  # Name of the save (e.g., checkpoint name)
        "patience": 500,  # Patience for early stopping
        "train/val/test": {
            "train": 0.8,  # Percentage of data used for training
            "val": 0,  # Percentage of data used for validation
            "test": 0.2  # Percentage of data used for testing
        }
    }
}

In [52]:
import pandas as pd
import shutil
import zipfile
def report(csv_name,true_array, pred_array):
    """
    Generates a classification report based on the true and predicted arrays for a given utility.

    Args:
        csv_name: Name of the .csv file.
        true_array: Array of true labels.
        pred_array: Array of predicted labels.

    Returns:
        Saves the report as a CSV file.
    """
    print(csv_name)
    report = classification_report(true_array, pred_array, output_dict=True)
    df = pd.DataFrame(report).transpose()
    file_name = 'results_tssb_MTF_'+str(Config["model"]["SEED"])+'_Without_linear/'+csv_name
    
    # Create the directory if it doesn't exist
    directory = os.path.dirname(file_name)
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    df.to_csv(file_name + ".csv")
    shutil.make_archive(file_name, 'zip', file_name)

In [ ]:
"""
This for loop performs the main function for graph generation, training, and saving the test results to a csv file.
If necessary, the peaks are also determined to predict the exact time of occurrence of the segmentation
"""

#Most common Config parameters
Config["graph"]["type"] = "MTF"
Config["graph"]["MTF"]["num_bins"] = 'auto'
Config["model"]["learning_rate"]= 0.0005
Config["model"]["SEED"] = 820
Config["model"]["range_epoch"]= 2000
for utility in range(75): # loop for all different tssb time series 

    #For generating, training and testing the graph
    main()

    # Generate a classification report for the utility
    report(get_versions_TSSB()[utility][:-4],true_array, pred_array)

    # If needed
    # Plot the predicted segmentation for the selected utility
    plt_peaks()